In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [16]:
# Load ratings data
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/movielens/u.data', sep='\t', names=column_names)

# Load movie titles
movie_titles = pd.read_csv('../data/movielens/u.item', sep='|', encoding='latin-1',
                           usecols=[0, 1], names=['item_id', 'title'])

# Merge the datasets
data = pd.merge(ratings, movie_titles, on='item_id')


In [17]:
data['interaction'] = data['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [18]:
user_ids = data['user_id'].unique().tolist()
item_ids = data['item_id'].unique().tolist()

user_id_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
item_id_to_idx = {item_id: idx for idx, item_id in enumerate(item_ids)}

data['user_idx'] = data['user_id'].map(user_id_to_idx)
data['item_idx'] = data['item_id'].map(item_id_to_idx)


In [19]:
train_data, test_data = train_test_split(
    data[['user_idx', 'item_idx', 'interaction']], test_size=0.2, random_state=42)


In [20]:
num_users = len(user_ids)
num_items = len(item_ids)
embedding_size = 32  # Adjustable based on experimentation


In [21]:
# User input and embedding
user_input = keras.Input(shape=(1,), name='user_input')
user_embedding = layers.Embedding(num_users, embedding_size, name='user_embedding')(user_input)
user_embedding = layers.Flatten()(user_embedding)

# Item input and embedding
item_input = keras.Input(shape=(1,), name='item_input')
item_embedding = layers.Embedding(num_items, embedding_size, name='item_embedding')(item_input)
item_embedding = layers.Flatten()(item_embedding)

# Concatenate user and item embeddings
concat = layers.Concatenate()([user_embedding, item_embedding])

# MLP layers
dense = layers.Dense(128, activation='relu')(concat)
dense = layers.Dense(64, activation='relu')(dense)
dense = layers.Dense(32, activation='relu')(dense)

# Output layer
output = layers.Dense(1, activation='sigmoid')(dense)

# Define the model
ncf_model = keras.Model(inputs=[user_input, item_input], outputs=output)

# Compile the model
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
from keras_tuner import HyperModel
from keras_tuner import RandomSearch

def build_model(hp):
    # User input and embedding
    user_input = keras.Input(shape=(1,), name='user_input')
    user_embedding = layers.Embedding(num_users, hp.Int('embedding_size', min_value=16, max_value=64, step=16))(user_input)
    user_embedding = layers.Flatten()(user_embedding)

    # Item input and embedding
    item_input = keras.Input(shape=(1,), name='item_input')
    item_embedding = layers.Embedding(num_items, hp.Int('embedding_size', min_value=16, max_value=64, step=16))(item_input)
    item_embedding = layers.Flatten()(item_embedding)

    # Concatenate user and item embeddings
    concat = layers.Concatenate()([user_embedding, item_embedding])

    # MLP layers with hyperparameters for dense units and dropout rate
    dense = layers.Dense(hp.Int('units_1', min_value=64, max_value=256, step=64), activation='relu')(concat)
    dense = layers.Dropout(hp.Float('dropout_1', 0.1, 0.5, step=0.1))(dense)
    dense = layers.Dense(hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu')(dense)
    dense = layers.Dropout(hp.Float('dropout_2', 0.1, 0.5, step=0.1))(dense)

    # Output layer
    output = layers.Dense(1, activation='sigmoid')(dense)

    model = keras.Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [25]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='movie_recommender'
)

# Run the tuner search
tuner.search(
    [train_user, train_item],
    train_label,
    validation_data=([test_user, test_item], test_label),
    batch_size=256,
    epochs=20,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)


Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.7127500176429749

Best val_accuracy So Far: 0.7127500176429749
Total elapsed time: 00h 00m 54s


In [26]:
# Retrieve the best model from the tuner
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model with early stopping
history = best_model.fit(
    [train_user, train_item],
    train_label,
    batch_size=256,
    epochs=50,
    validation_data=([test_user, test_item], test_label),
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)


Epoch 1/50


C:\Users\green\anaconda3\envs\NeuralCollab\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7435 - loss: 0.5113 - val_accuracy: 0.7136 - val_loss: 0.5596
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7482 - loss: 0.5065 - val_accuracy: 0.7111 - val_loss: 0.5616
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7551 - loss: 0.4969 - val_accuracy: 0.7142 - val_loss: 0.5642
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7617 - loss: 0.4876 - val_accuracy: 0.7110 - val_loss: 0.5713


In [27]:
loss, accuracy = best_model.evaluate([test_user, test_item], test_label)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.7107 - loss: 0.5695
Test Loss: 0.5713, Test Accuracy: 0.7110


In [30]:
def get_liked_movies(user_id, num_movies=10):
    user_data = data[(data['user_id'] == user_id) & (data['rating'] >= 4)]
    liked_movies = user_data.sample(n=min(num_movies, len(user_data)))['title'].tolist()
    return liked_movies

def get_disliked_movies(user_id, num_movies=10):
    user_data = data[(data['user_id'] == user_id) & (data['rating'] <= 2)]
    disliked_movies = user_data.sample(n=min(num_movies, len(user_data)))['title'].tolist()
    return disliked_movies


In [31]:
def recommend_movies(user_id, num_recommendations=10):
    user_idx = user_id_to_idx.get(user_id)
    if user_idx is None:
        print("User ID not found.")
        return []
    
    # Items the user has interacted with
    user_data = data[data['user_idx'] == user_idx]
    interacted_items = set(user_data['item_idx'].tolist())
    
    # Items not yet interacted with
    all_items = set(range(num_items))
    items_to_predict = list(all_items - interacted_items)
    
    # Predict interaction scores
    user_array = np.full(len(items_to_predict), user_idx)
    item_array = np.array(items_to_predict)
    
    predictions = best_model.predict([user_array, item_array], batch_size=1024).flatten()
    
    # Get top N items
    top_indices = predictions.argsort()[-num_recommendations:][::-1]
    recommended_item_idxs = [items_to_predict[i] for i in top_indices]
    
    # Map item indices to titles
    recommended_item_ids = [item_ids[idx] for idx in recommended_item_idxs]
    recommended_titles = movie_titles[movie_titles['item_id'].isin(recommended_item_ids)]['title'].tolist()
    
    return recommended_titles


In [35]:
import random

# Choose a random user ID from the dataset
random_user_id = random.choice(user_ids)

# Get liked, disliked, and recommended movies
liked_movies = get_liked_movies(random_user_id, num_movies=10)
disliked_movies = get_disliked_movies(random_user_id, num_movies=10)
recommended_movies = recommend_movies(random_user_id, num_recommendations=10)

# Display the final report
print(f"Final Report for User {random_user_id}:")
print("\nMovies They Liked:")
for idx, title in enumerate(liked_movies, 1):
    print(f"{idx}. {title}")

print("\nMovies They Didn't Like:")
for idx, title in enumerate(disliked_movies, 1):
    print(f"{idx}. {title}")

print("\nRecommended Movies They Might Like:")
for idx, title in enumerate(recommended_movies, 1):
    print(f"{idx}. {title}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Final Report for User 306:

Movies They Liked:
1. Marvin's Room (1996)
2. Michael Collins (1996)
3. Cold Comfort Farm (1995)
4. Stealing Beauty (1996)
5. English Patient, The (1996)
6. Kolya (1996)
7. Fargo (1996)
8. Secrets & Lies (1996)
9. Mighty Aphrodite (1995)
10. Antonia's Line (1995)

Movies They Didn't Like:
1. Contact (1997)
2. Last Supper, The (1995)
3. Grumpier Old Men (1995)

Recommended Movies They Might Like:
1. Whole Wide World, The (1996)
2. Innocents, The (1961)
3. When We Were Kings (1996)
4. Fresh (1994)
5. Maybe, Maybe Not (Bewegte Mann, Der) (1994)
6. Faust (1994)
7. Mina Tannenbaum (1994)
8. Stonewall (1995)
9. Pather Panchali (1955)
10. Bitter Sugar (Azucar Amargo) (1996)


In [36]:
# Save the best model
best_model.save('../app/models/ncf_model.h5')


In [37]:
import pickle

# Save user_id_to_idx mapping
with open('../app/models/user_id_to_idx.pkl', 'wb') as f:
    pickle.dump(user_id_to_idx, f)

# Save item_ids list (index corresponds to item_idx)
with open('../app/models/item_ids.pkl', 'wb') as f:
    pickle.dump(item_ids, f)

# Save item_id_to_title mapping
item_id_to_title = dict(zip(movie_titles['item_id'], movie_titles['title']))
with open('../app/models/item_id_to_title.pkl', 'wb') as f:
    pickle.dump(item_id_to_title, f)

# Save data DataFrame (if needed for liked/disliked movies)
data.to_pickle('../app/models/data.pkl')


In [38]:
# Save the list of user IDs
with open('../app/models/user_ids.pkl', 'wb') as f:
    pickle.dump(user_ids, f)
